In [1]:
#Import libraries
import json
from pandas import json_normalize
import pandas as pd
import numpy as np
from sentinelhub import SHConfig
from sentinelhub import WmsRequest, WcsRequest, MimeType, CRS, BBox, CustomUrlParam
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

## Import data


In [2]:
#Import json and convert into dataframe
with open('inputs\\farm_map.json') as f:
    data = json.load(f)
df_data = json_normalize(data["features"]) 
df_data

,type,geometry.type,geometry.coordinates,properties.values,properties.unit.state.area,properties.unit.state.year_of_planting,properties.unit.state.variety,properties.unit.metadata,properties.unit.id,properties.unit.full_name,properties.unit.name
0,Feature,Polygon,"[[[-75.6396578867764, -14.199701335951724], [-...","[{'value': 3581.158766439885, 'variety': {}, '...",6.530601,[2019],"[hass, fuerte, zutano]","[{'level_name': 'empresa', 'name': 'agricola c...",90214,lote AGRICOLA CHAPI.DON ERNESTO.PAL-1.36,AGRICOLA CHAPI.DON ERNESTO.PAL-1.36
1,Feature,Polygon,"[[[-75.64231663022875, -14.20062882414364], [-...","[{'value': 4037.3751791811173, 'variety': {}, ...",6.555069,[2019],"[hass, fuerte, zutano]","[{'level_name': 'empresa', 'name': 'agricola c...",90215,lote AGRICOLA CHAPI.DON ERNESTO.PAL-1.37,AGRICOLA CHAPI.DON ERNESTO.PAL-1.37
2,Feature,Polygon,"[[[-75.63898926117189, -14.201527268326426], [...","[{'value': 4658.420440484501, 'variety': {}, '...",6.533254,[2019],"[hass, fuerte, zutano]","[{'level_name': 'empresa', 'name': 'agricola c...",90216,lote AGRICOLA CHAPI.DON ERNESTO.PAL-1.42,AGRICOLA CHAPI.DON ERNESTO.PAL-1.42
3,Feature,Polygon,"[[[-75.64164955317383, -14.202454024468352], [...","[{'value': 4196.272165747293, 'variety': {}, '...",6.551013,[2019],"[hass, fuerte, zutano]","[{'level_name': 'empresa', 'name': 'agricola c...",90217,lote AGRICOLA CHAPI.DON ERNESTO.PAL-1.43,AGRICOLA CHAPI.DON ERNESTO.PAL-1.43
4,Feature,Polygon,"[[[-75.64164955317383, -14.202454024468352], [...","[{'value': 4744.580970357842, 'variety': {}, '...",6.534646,[2019],"[hass, fuerte, zutano]","[{'level_name': 'empresa', 'name': 'agricola c...",90218,lote AGRICOLA CHAPI.DON ERNESTO.PAL-1.48,AGRICOLA CHAPI.DON ERNESTO.PAL-1.48
5,Feature,Polygon,"[[[-75.64431888131925, -14.203383890611855], [...","[{'value': 4914.518933170159, 'variety': {}, '...",6.562506,[2019],"[hass, fuerte, zutano]","[{'level_name': 'empresa', 'name': 'agricola c...",90219,lote AGRICOLA CHAPI.DON ERNESTO.PAL-1.49,AGRICOLA CHAPI.DON ERNESTO.PAL-1.49
6,Feature,Polygon,"[[[-75.63765710540375, -14.20516513033943], [-...","[{'value': 3081.5154889943, 'variety': {}, 'me...",6.498691,[2019],"[hass, fuerte, zutano]","[{'level_name': 'empresa', 'name': 'agricola c...",90221,lote AGRICOLA CHAPI.DON ERNESTO.PAL-2.54,AGRICOLA CHAPI.DON ERNESTO.PAL-2.54
7,Feature,Polygon,"[[[-75.64031869971863, -14.206095306337506], [...","[{'value': 2970.0807760737507, 'variety': {}, ...",6.525776,[2019],"[hass, fuerte, zutano]","[{'level_name': 'empresa', 'name': 'agricola c...",90222,lote AGRICOLA CHAPI.DON ERNESTO.PAL-2.55,AGRICOLA CHAPI.DON ERNESTO.PAL-2.55
8,Feature,Polygon,"[[[-75.64031869971863, -14.206095306337506], [...","[{'value': 4789.354613313465, 'variety': {}, '...",6.561207,[2019],"[hass, fuerte, zutano]","[{'level_name': 'empresa', 'name': 'agricola c...",90224,lote AGRICOLA CHAPI.DON ERNESTO.PAL-3.60,AGRICOLA CHAPI.DON ERNESTO.PAL-3.60
9,Feature,Polygon,"[[[-75.64298986878343, -14.20702879053244], [-...","[{'value': 4222.732171703558, 'variety': {}, '...",6.569781,[2019],"[hass, fuerte, zutano]","[{'level_name': 'empresa', 'name': 'agricola c...",90225,lote AGRICOLA CHAPI.DON ERNESTO.PAL-3.61,AGRICOLA CHAPI.DON ERNESTO.PAL-3.61


In [ ]:
#Get important variables as columns
df_data['Kg/Ha'] = df_data["properties.values"].apply(lambda x: x[0]["value"])
df_data['Kg/Ha_date'] = df_data["properties.values"].apply(lambda x: x[0]["date_stamp"])

In [3]:
#Import dates excel
df = pd.read_excel("inputs\\SentinelHubImage-available_dates.xlsx")
df = df.sort_values("_datetime_utc", ascending = True)
df

,id,farm,_date,_datetime_utc,validated
88,8519,163,2016-07-15,2016-07-15 15:23:58,1
87,8518,163,2016-09-13,2016-09-13 15:26:48,1
86,8515,163,2016-11-22,2016-11-22 15:19:30,1
85,8514,163,2016-12-02,2016-12-02 15:21:03,1
84,8513,163,2016-12-22,2016-12-22 15:23:53,1
...,...,...,...,...,...
4,10946,163,2020-02-10,2020-02-10 15:26:52,1
3,11202,163,2020-02-15,2020-02-15 15:26:51,1
2,11899,163,2020-03-11,2020-03-11 15:26:55,1
1,13892,163,2020-04-15,2020-04-15 15:26:57,1


# Data wrangling - format

In [4]:
#Give the polygon coordinates BBox format and Geometry Url Polygon format
#To check Polygon format: https://github.com/sentinel-hub/sentinelhub-py/issues/37
def poly_to_format(x):
    lons = []
    lats = []
    for i in x[0]:
        lons += [i[0]]
        lats += [i[1]]
    
    box = [max(lons), max(lats), min(lons), min(lats)]
    
    poly = "POLYGON(("
    for lon, lat in zip(lons, lats):
        lon_lat = str(lon) +" "+ str(lat) + ", "
        poly += lon_lat     
    poly = poly[0:-2]
    poly += "))"
      
    return(box, poly)   

In [5]:
#Apply function to pass coordinates to BBox format and Polygon format
df_data["bbox_coordinates"] = df_data["geometry.coordinates"].apply(lambda x: poly_to_format(x)[0])
df_data["poly_coordinates"] = df_data["geometry.coordinates"].apply(lambda x: poly_to_format(x)[1])
df_data[["bbox_coordinates", "poly_coordinates"]]

,bbox_coordinates,poly_coordinates
0,"[-75.6396578867764, -14.197876401443217, -75.6...",POLYGON((-75.6396578867764 -14.199701335951724...
1,"[-75.64231663022875, -14.198802571721302, -75....",POLYGON((-75.64231663022875 -14.20062882414364...
2,"[-75.63898926117189, -14.199701335951724, -75....",POLYGON((-75.63898926117189 -14.20152726832642...
3,"[-75.64164955317383, -14.20062882414364, -75.6...",POLYGON((-75.64164955317383 -14.20245402446835...
4,"[-75.63832124398837, -14.201527268326426, -75....",POLYGON((-75.64164955317383 -14.20245402446835...
5,"[-75.64098212423809, -14.202454024468352, -75....",POLYGON((-75.64431888131925 -14.20338389061185...
6,"[-75.63765710540375, -14.203351509792409, -75....",POLYGON((-75.63765710540375 -14.20516513033943...
7,"[-75.64031869971863, -14.20428015816881, -75.6...",POLYGON((-75.64031869971863 -14.20609530633750...
8,"[-75.6369858615888, -14.20516513033943, -75.64...",POLYGON((-75.64031869971863 -14.20609530633750...
9,"[-75.63965014888008, -14.206095306337506, -75....",POLYGON((-75.64298986878343 -14.20702879053244...


# Start with Sentinel

In [6]:
#Open Sentinel instance
INSTANCE_ID = '13c0b719-d307-47fe-9472-666ef2332582'  

if INSTANCE_ID:
    config = SHConfig()
    config.instance_id = INSTANCE_ID
else:
    config = None


In [7]:
#Get array and average NDVI for a specific space-time picture

#NDVI layer calculation:
#var ndvi = (B08-B04)/(B08+B04);
#return [ndvi];

def get_mean_NDVI(box, date, poly):
    bbox = BBox(bbox=box, crs=CRS.WGS84)
    #Request NDVI layer for 500 width TIFF_d32f image
    NDVI_request = WmsRequest(
        layer='NDVI',
        bbox=bbox,
        time=date,
        width=500,
        custom_url_params={
                        CustomUrlParam.GEOMETRY: poly
                        },
        image_format = MimeType.TIFF_d32f,
        config=config
    )
    NDVIs = NDVI_request.get_data()
    #Create list with all NDVI values from image to calculate average
    ndvis = []
    for ndvi_list in NDVIs[0]:
        for ndvi in ndvi_list:
            #Ignore low values (not in field or damaged by clouds) and 1s (out of polygon)
            if (ndvi>0.2) & (ndvi<1):
                ndvis += [ndvi]
    #Case for images with all NDVI low values an 1, return nan for average
    if len(ndvis)>0:
        NDVI = sum(ndvis) / len(ndvis)
    else:
        NDVI = np.nan
        print(box, date)
        
    #Return image and average
    return(NDVIs, NDVI)

In [8]:
#Function for saving images

def save_image(image, path, factor=1):
    fig = plt.subplots(nrows=1, ncols=1, figsize=(15, 7))

    if np.issubdtype(image.dtype, np.floating):
        
        plt.imshow(np.minimum(image * factor, 1))
    else:
        plt.imshow(image)
    plt.savefig(path)
    plt.close()

In [9]:
#Get NDVI image and average values for date and coordinates
for date in tqdm(df["_date"]): 
    NVDI_arrays_list = []
    NDVIs_list = []
    print(date)
    
    #Iterate through bbox and polygon df rows
    for index, row in tqdm(df_data[["bbox_coordinates", "poly_coordinates", "properties.unit.id"]].iterrows()):
        box = row[0]
        poly = row[1]
        id_lote = row[2]
        NVDI, ndvi = get_mean_NDVI(box, date, poly)
        NVDI_arrays_list += NVDI
        NDVIs_list += [ndvi]
        
        #Save images
        image_path = "outputs\\images\\"+date+"_"+str(id_lote)+".png"
        save_image(NVDI[0], image_path)
        
    #Save NVDI arrays and averages in data dataframe
    df_data[date+"_array"] = NVDI_arrays_list
    df_data[date] = NDVIs_list
    
    
    
        

2016-07-15


[-75.64769678112165, -14.192480425330247, -75.65097557981476, -14.19515473241042] 2016-07-15
[-75.64166749773169, -14.192429748404132, -75.64495509648042, -14.195132802468311] 2016-07-15
[-75.64100407458932, -14.194213164268845, -75.64430202199843, -14.19694295233761] 2016-07-15

2016-09-13


[-75.64434649855319, -14.193355636804679, -75.64763941311051, -14.196068495618299] 2016-09-13
[-75.64769678112165, -14.192480425330247, -75.65097557981476, -14.19515473241042] 2016-09-13
[-75.64166749773169, -14.192429748404132, -75.64495509648042, -14.195132802468311] 2016-09-13
[-75.64701007809187, -14.194282863638566, -75.65031370091558, -14.19702703200722] 2016-09-13
[-75.64100407458932, -14.194213164268845, -75.64430202199843, -14.19694295233761] 2016-09-13
[-75.64368793366349, -14.19514832766311, -75.6469826482682, -14.197881148780363] 2016-09-13

2016-11-22


[-75.64769678112165, -14.192480425330247, -75.65097557981476, -14.19515473241042] 2016-11-22
[-75.64166749773169, -14.192429748404132, -75.64495509648042, -14.195132802468311] 2016-11-22
[-75.64701007809187, -14.194282863638566, -75.65031370091558, -14.19702703200722] 2016-11-22
[-75.64100407458932, -14.194213164268845, -75.64430202199843, -14.19694295233761] 2016-11-22
[-75.64570305607081, -14.1978903845171, -75.64898805011781, -14.200586462611307] 2016-11-22

2016-12-02


[-75.64234056627969, -14.190610001340819, -75.64561150025328, -14.193291049214046] 2016-12-02
[-75.64166749773169, -14.192429748404132, -75.64495509648042, -14.195132802468311] 2016-12-02
[-75.64701007809187, -14.194282863638566, -75.65031370091558, -14.19702703200722] 2016-12-02
[-75.64100407458932, -14.194213164268845, -75.64430202199843, -14.19694295233761] 2016-12-02
[-75.6463550733476, -14.196107720636288, -75.64964409685477, -14.198813508297691] 2016-12-02
[-75.64570305607081, -14.1978903845171, -75.64898805011781, -14.200586462611307] 2016-12-02

2016-12-22


...


2020-02-10



2020-02-15



2020-03-11



2020-04-15



2020-04-20


# Saving raw outputs

In [10]:
df_data

,type,geometry.type,geometry.coordinates,properties.values,properties.unit.state.area,properties.unit.state.year_of_planting,properties.unit.state.variety,properties.unit.metadata,properties.unit.id,properties.unit.full_name,...,2020-02-10_array,2020-02-10,2020-02-15_array,2020-02-15,2020-03-11_array,2020-03-11,2020-04-15_array,2020-04-15,2020-04-20_array,2020-04-20
0,Feature,Polygon,"[[[-75.6396578867764, -14.199701335951724], [-...","[{'value': 3581.158766439885, 'variety': {}, '...",6.530601,[2019],"[hass, fuerte, zutano]","[{'level_name': 'empresa', 'name': 'agricola c...",90214,lote AGRICOLA CHAPI.DON ERNESTO.PAL-1.36,...,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.594447,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.585075,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.587418,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.623925,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.649108
1,Feature,Polygon,"[[[-75.64231663022875, -14.20062882414364], [-...","[{'value': 4037.3751791811173, 'variety': {}, ...",6.555069,[2019],"[hass, fuerte, zutano]","[{'level_name': 'empresa', 'name': 'agricola c...",90215,lote AGRICOLA CHAPI.DON ERNESTO.PAL-1.37,...,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.594103,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.587701,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.585747,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.622790,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.650104
2,Feature,Polygon,"[[[-75.63898926117189, -14.201527268326426], [...","[{'value': 4658.420440484501, 'variety': {}, '...",6.533254,[2019],"[hass, fuerte, zutano]","[{'level_name': 'empresa', 'name': 'agricola c...",90216,lote AGRICOLA CHAPI.DON ERNESTO.PAL-1.42,...,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.608672,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.593903,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.600877,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.628707,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.660342
3,Feature,Polygon,"[[[-75.64164955317383, -14.202454024468352], [...","[{'value': 4196.272165747293, 'variety': {}, '...",6.551013,[2019],"[hass, fuerte, zutano]","[{'level_name': 'empresa', 'name': 'agricola c...",90217,lote AGRICOLA CHAPI.DON ERNESTO.PAL-1.43,...,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.607753,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.595797,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.601193,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.622388,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.655480
4,Feature,Polygon,"[[[-75.64164955317383, -14.202454024468352], [...","[{'value': 4744.580970357842, 'variety': {}, '...",6.534646,[2019],"[hass, fuerte, zutano]","[{'level_name': 'empresa', 'name': 'agricola c...",90218,lote AGRICOLA CHAPI.DON ERNESTO.PAL-1.48,...,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.620011,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.603345,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.614018,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.634250,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.672965
5,Feature,Polygon,"[[[-75.64431888131925, -14.203383890611855], [...","[{'value': 4914.518933170159, 'variety': {}, '...",6.562506,[2019],"[hass, fuerte, zutano]","[{'level_name': 'empresa', 'name': 'agricola c...",90219,lote AGRICOLA CHAPI.DON ERNESTO.PAL-1.49,...,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.624000,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.606458,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.610833,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.628968,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0.665730
6,Feature,Polygon,"[[[-75.63765710540375, -14.20516513033943], [-...","[{'value': 3081.5154889943, 'variety': {}, 'me...",6.498691,[2019],"[hass, fuerte, zutano]","[{'level_name': 'empresa', 'name': 'agric

In [13]:
df_data.describe()

,properties.unit.state.area,properties.unit.id,2016-07-15,2016-09-13,2016-11-22,2016-12-02,2016-12-22,2017-01-01,2017-02-20,2017-04-01,...,2019-11-17,2019-12-07,2019-12-17,2019-12-27,2020-02-05,2020-02-10,2020-02-15,2020-03-11,2020-04-15,2020-04-20
count,33.000000,33.000000,30.000000,27.000000,28.000000,27.000000,27.000000,26.000000,24.000000,24.000000,...,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000
mean,6.434269,90232.909091,0.455755,0.426877,0.507662,0.489531,0.469888,0.474038,0.519918,0.520639,...,0.460241,0.453779,0.476877,0.485428,0.465598,0.491843,0.482191,0.486950,0.514967,0.537901
std,0.119440,11.336786,0.190395,0.143901,0.221135,0.193776,0.180455,0.171250,0.173757,0.166001,...,0.166364,0.145905,0.146822,0.135946,0.118699,0.133040,0.127277,0.124192,0.114048,0.130466
min,6.191944,90214.000000,0.203571,0.201993,0.203846,0.201364,0.203394,0.202460,0.200854,0.217155,...,0.228292,0.237624,0.245169,0.267559,0.261637,0.269381,0.268644,0.285139,0.337190,0.324875
25%,6.330826,90224.000000,0.223614,0.246096,0.221436,0.235692,0.232803,0.243460,0.507172,0.501256,...,0.283129,0.317854,0.345917,0.367561,0.352991,0.367120,0.363346,0.377239,0.420472,0.423330
50%,6.469184,90233.000000,0.597329,0.522298,0.660009,0.615734,0.588359,0.581606,0.614721,0.612547,...,0.583068,0.556322,0.576954,0.574817,0.545701,0.581051,0.560072,0.561151,0.582440,0.617909
75%,6.536344,90242.000000,0.608997,0.531056,0.672041,0.629245,0.600378,0.591623,0.624135,0.620913,...,0.607893,0.580026,0.603445,0.600299,0.569329,0.608672,0.594334,0.600877,0.616189,0.652566
max,6.575069,90251.000000,0.634761,0.543809,0.686018,0.644637,0.614079,0.604310,0.633505,0.629007,...,0.647146,0.623079,0.646210,0.639688,0.587165,0.632569,0.620516,0.622821,0.639436,0.680620


In [28]:
df_data.to_csv("outputs\\df_data.csv", index = False)